In [1]:
# Library imports
import numpy as np
import pandas as pd
import csv
import pickle
from tqdm.autonotebook import tqdm

F:\Anaconda\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
# Read data from data/four_area
author_df = pd.read_csv("data/four_area/author.txt", sep = "\t", names=["ID", "Author name"],encoding='utf8')
conf_df = pd.read_csv("data/four_area/conf.txt", sep = "\t", names=["ID", "Conference name"])
paper_df = pd.read_csv("data/four_area/paper.txt", sep = "\t", names=["ID", "Paper title"])
term_df = pd.read_csv("data/four_area/term.txt", sep = "\t", names=["ID", "Term"])
paper_author = pd.read_csv("data/four_area/paper_author.txt", sep = "\t", names=["paperID", "authorID"])
paper_conf = pd.read_csv("data/four_area/paper_conf.txt", sep = "\t", names=["paperID", "confID"])
paper_term = pd.read_csv("data/four_area/paper_term.txt", sep = "\t", names=["paperID", "termID"])

### ID in DBLP_four_area are differents from four_area. four_area's ID are rehashed.
### Label are defined as follows
* Database->1
* Data Mining->2
* Machine Learning->3
* Information Retrieval->4

In [3]:
# Read data from data/DBLP_four_area
author_dict = pd.read_csv("data/DBLP_four_area/cleaned_author_dict.txt", sep = "\t", names=["ID", "Author name"], encoding='utf8')
conf_dict = pd.read_csv("data/DBLP_four_area/conf_dict.txt", sep = "\t", names=["ID", "Conference name"])
term_dict = pd.read_csv("data/DBLP_four_area/term_dict.txt", sep = "\t", names=["ID", "Term"])
author_label = pd.read_csv("data/DBLP_four_area/author_label.txt", sep = "\t", names=["authorID", "Label"])
conf_label = pd.read_csv("data/DBLP_four_area/conf_label.txt", sep = "\t", names=["confID", "Conference name", "Label"])

In [4]:
# Merge
conf_dict_m = pd.merge(conf_dict, conf_df, on='Conference name')
author_dict_m = pd.merge(author_dict, author_df, on='Author name')

In [5]:
print ("Number of nodes %d"%(author_df.shape[0] + conf_df.shape[0] + paper_df.shape[0] + term_df.shape[0]))
print ("Number of edges %d"%(paper_author.shape[0] + paper_conf.shape[0]+paper_term.shape[0]) )

Number of nodes 70647
Number of edges 332388


### WTF are we doing here
* Define G = (V, E, YL, W), where G stands for graph, V is the set of all vertices, E is the set of all edges, YL is the set of observed labels, and W is the weight.
* V size is 70647. E size is 332388.
* Our goal is to find a function f that maps G to Y. In other words, we want to know a prediction function that, given G, can predict the label Y.
* New Param = old param + gradient * learning rate
* Gradiant can be expressed explicitly in a formula, that requires LBP to get expectation. 
* We need a way to evaluate the performance

* What's the percentage of unobserved labels? 
* (author_label.shape[0] / author.shape[0] * 100) = 14.758553410912132%

# 1 Hyperparameters

In [6]:
learning_rate = 0.01
learning_rates = [0.001, 0.01, 0.1] 
learning_iteration = 100
learning_iterations = [50, 100, 200]

# 2 Setup

In [20]:
author_matrix = pd.DataFrame(np.zeros(shape=(28702,28702)),
                             columns=author_dict['ID'].unique(), 
                             index=author_dict['ID'].unique())

In [8]:
conf_matrix= pd.DataFrame(np.zeros(shape=(28702,20)),
                             columns=conf_dict['ID'].unique(), 
                             index=author_dict['ID'].unique())

In [21]:
# Brute Force

for paper in tqdm(paper_author['paperID']):
    authors = list(paper_author[paper_author['paperID'] == paper]['authorID'])
    confID = int(paper_conf[paper_conf['paperID']==paper]['confID'].to_string(index=False).strip())
    if (len(authors) > 1) :
        for i in range(len(authors)):
            authorID1 = authors[i]
            authorDictID1 = author_dict_m[author_dict_m["ID_y"] == authorID1]['ID_x'].to_string(index=False).strip()        
            confDictID = conf_dict_m[conf_dict_m['ID_y']==confID]['ID_x'].to_string(index=False).strip()
            conf_matrix.at[int(authorDictID1),int(confDictID)] += 1
            for j in range(i+1, len(authors)):
                authorID2 = authors[j]
                authorDictID2 = author_dict_m[author_dict_m["ID_y"] == authorID2]['ID_x'].to_string(index=False).strip()
                author_matrix.at[int(authorDictID1),int(authorDictID2)] += 1
                author_matrix.at[int(authorDictID2),int(authorDictID1)] += 1
                
# author_matrix = author_matrix.astype('int16')
# author_matrix.to_pickle('./author_matrix.pickle')

# conf_matrix = conf_matrix.astype('int16')
# conf_matrix.to_pickle('./conf_matrix.pickle')

In [13]:
# Read
author_matrix = pd.read_pickle('./author_matrix.pickle')
conf_matrix = pd.read_pickle('./conf_matrix.pickle')

In [24]:
author_matrix

,1,2,3,4,5,6,7,8,9,10,...,28693,28694,28695,28696,28697,28698,28699,28700,28701,28702
1,0,100,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13,0,0,17,19,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,17,0,22,0,0,0,10,13,...,0,0,0,0,0,0,0,0,0,0
5,0,0,19,22,0,85,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28700,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
